# Задание 1
Скачайте с сайта [сайта](https://grouplens.org/datasets/movielens/) датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

## download and unzip dataset in use....

In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2022-01-06 21:46:25--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  48.6MB/s    in 5.6s    

2022-01-06 21:46:31 (44.9 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [2]:
!unzip /content/ml-25m.zip 

Archive:  /content/ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


## Analysis dataset
 

In [3]:
import pandas as pd

##  1. let's look into rating.csv file 

In [4]:
rating_df = pd.read_csv('/content/ml-25m/ratings.csv')
rating_df.tail()

,userId,movieId,rating,timestamp
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434
25000094,162541,63876,5.0,1240952515


In [5]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


evaluate whether any value is missed

In [6]:
def are_some_values_missed(df):
  if df.isnull().values.any(): 
    print('the dataset has some missed values')
  else:
    print('the dataset does not have any missed values')

In [7]:
are_some_values_missed(rating_df)

the dataset does not have any missed values


## let's look into movies.csv file

In [8]:
movies_df = pd.read_csv('/content/ml-25m/movies.csv')

In [9]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


evaluate whether any value is missed

In [11]:
are_some_values_missed(movies_df)

the dataset does not have any missed values


## filtration

In [12]:
five_star_rating_df = rating_df[rating_df['rating'] == 5]
five_star_rating_df.head(10)
five_star_rating_df.drop(["userId","timestamp"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
five_star_rating_df.head()

,movieId,rating
0,296,5.0
2,307,5.0
3,665,5.0
8,1237,5.0
18,2632,5.0


## agregation

In [15]:
aggregated_five_start_rating = five_star_rating_df.groupby(by='movieId').sum()
aggregated_five_start_rating.head(10)

,rating
movieId,
1,67530.0
2,9435.0
3,5195.0
4,855.0
5,4200.0
6,26095.0
7,6935.0
8,555.0
9,1060.0


## sorting

In [16]:
aggregated_five_start_rating.sort_values(by ='rating',inplace=True, ascending=False)
aggregated_five_start_rating.head(10)

,rating
movieId,
318,197765.0
296,160845.0
356,129590.0
260,129020.0
2571,127410.0
527,124265.0
593,124005.0
858,122090.0
50,107925.0


# join two tables to figure out the name of the film

In [18]:
joined_tables = aggregated_five_start_rating.join(movies_df.set_index('movieId'), on='movieId', how='left')
joined_tables.head()

,rating,title,genres
movieId,,,
318,197765.0,"Shawshank Redemption, The (1994)",Crime|Drama
296,160845.0,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
356,129590.0,Forrest Gump (1994),Comedy|Drama|Romance|War
260,129020.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
2571,127410.0,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [19]:
film_name = joined_tables['title'].iloc[0]
film_name

'Shawshank Redemption, The (1994)'

**Ответ на 1 задание -> Shawshank Redemption, The (1994)**

Для интереса рассмотрим какие оценки и в каком количестве были поставлены фильму из задания 1


In [20]:
rating_df = pd.read_csv('/content/ml-25m/ratings.csv')


In [21]:
rating_df = rating_df.groupby(by=['movieId', 'rating']).agg({'userId':'count'}).rename(columns={'userId':'Total_Numbers'})
rating_df.head(10)

Total_Numbers
movieId rating               
1       0.5               406
        1.0               665
        1.5               395
        2.0              1734
        2.5              1459
        3.0              9463
        3.5              5478
        4.0             18874
        4.5              5329
        5.0             13506

In [22]:
joined_tables = rating_df.join(movies_df.set_index('movieId'), on='movieId', how='left')
joined_tables.head(10)


Total_Numbers  ...                                       genres
movieId rating                 ...                                             
1       0.5               406  ...  Adventure|Animation|Children|Comedy|Fantasy
        1.0               665  ...  Adventure|Animation|Children|Comedy|Fantasy
        1.5               395  ...  Adventure|Animation|Children|Comedy|Fantasy
        2.0              1734  ...  Adventure|Animation|Children|Comedy|Fantasy
        2.5              1459  ...  Adventure|Animation|Children|Comedy|Fantasy
        3.0              9463  ...  Adventure|Animation|Children|Comedy|Fantasy
        3.5              5478  ...  Adventure|Animation|Children|Comedy|Fantasy
        4.0             18874  ...  Adventure|Animation|Children|Comedy|Fantasy
        4.5              5329  ...  Adventure|Animation|Children|Comedy|Fantasy
        5.0             13506  ...  Adventure|Animation|Children|Comedy|Fantasy

[10 rows x 3 columns]

In [23]:
joined_tables[joined_tables['title'] == film_name]

Total_Numbers                             title       genres
movieId rating                                                              
318     0.5               286  Shawshank Redemption, The (1994)  Crime|Drama
        1.0               353  Shawshank Redemption, The (1994)  Crime|Drama
        1.5               193  Shawshank Redemption, The (1994)  Crime|Drama
        2.0               819  Shawshank Redemption, The (1994)  Crime|Drama
        2.5               726  Shawshank Redemption, The (1994)  Crime|Drama
        3.0              4703  Shawshank Redemption, The (1994)  Crime|Drama
        3.5              3888  Shawshank Redemption, The (1994)  Crime|Drama
        4.0             18836  Shawshank Redemption, The (1994)  Crime|Drama
        4.5             12125  Shawshank Redemption, The (1994)  Crime|Drama
        5.0             39553  Shawshank Redemption, The (1994)  Crime|Drama

# Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [24]:
!wget https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15255/10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip

--2022-01-06 21:51:56--  https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15255/10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip
Resolving u.netology.ngcdn.ru (u.netology.ngcdn.ru)... 104.22.41.115, 104.22.40.115, 172.67.26.246, ...
Connecting to u.netology.ngcdn.ru (u.netology.ngcdn.ru)|104.22.41.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4762826 (4.5M) [application/x-zip-compressed]
Saving to: ‘10._Основы_pandas.zip’

10._Основы_pandas.z 100%[===================>]   4.54M   462KB/s    in 9.9s    

2022-01-06 21:52:07 (472 KB/s) - ‘10._Основы_pandas.zip’ saved [4762826/4762826]



In [25]:
!unzip /content/10._Основы_pandas.zip

Archive:  /content/10._Основы_pandas.zip
  inflating: power.csv               
  inflating: transactions.csv        


In [26]:
power_df = pd.read_csv('/content/power.csv')
power_df.head(10)

,country,year,quantity,category
0,Austria,1996,5.0,1
1,Austria,1995,17.0,1
2,Belgium,2014,0.0,1
3,Belgium,2013,0.0,1
4,Belgium,2012,35.0,1
5,Belgium,2011,25.0,1
6,Belgium,2010,22.0,1
7,Belgium,2009,45.0,1
8,Czechia,1998,1.0,1
9,Czechia,1995,7.0,1


In [27]:
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189482 entries, 0 to 1189481
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   country   1189482 non-null  object 
 1   year      1189482 non-null  int64  
 2   quantity  1189482 non-null  float64
 3   category  1189482 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 36.3+ MB


In [28]:
are_some_values_missed(power_df)

the dataset does not have any missed values


# Analysis


Row filter

In [29]:
Latvia = power_df['country'] == 'Latvia'
Lithuania = power_df['country'] == 'Lithuania'
Estonia = power_df['country'] == 'Estonia'
year_more_2005 = power_df['year'] >= 2005
year_less_2010 = power_df['year'] <= 2010
category_4 = power_df['category'] == 4
category_12 = power_df['category'] == 12
category_21 = power_df['category'] == 21
positive_quantity = power_df['quantity'] >= 0

baltics_power_df = power_df[(Latvia | Lithuania | Estonia) &
                            year_more_2005 & year_less_2010 &
                            (category_4 | category_12 | category_21 ) &
                            positive_quantity].reset_index(0)
baltics_power_df.describe()


,index,year,quantity,category
count,408.000000,408.000000,408.000000,408.000000
mean,169106.892157,2007.575980,589.656863,16.367647
std,80981.910531,1.622662,1063.388571,6.429136
min,10617.000000,2005.000000,0.000000,4.000000
25%,109039.750000,2006.000000,1.000000,12.000000
50%,210580.500000,2008.000000,27.000000,21.000000
75%,228613.250000,2009.000000,760.000000,21.000000
max,253986.000000,2010.000000,4909.000000,21.000000


In [30]:
baltics_power_df['quantity'].sum()

240580.0

Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):

[link text1](https://fortrader.org/quotes)

[link text2](https://www.finanz.ru/valyuty/v-realnom-vremeni)

In [31]:
import requests # стандартный инструментом для составления HTTP-запросов 
from bs4 import BeautifulSoup

In [32]:

URL = 'https://fortrader.org/quotes'  
HEADERS ={'user-agent': 'Mozilla/5.0'}   

def get_html(url, headers, params=None):
    response = requests.get(url, headers)

    if response.status_code == 200:
      html = BeautifulSoup(response.text, 'html.parser')
    else: 
      html = '<page is not awailable>'
    return html

soup = get_html(url = URL, headers = HEADERS)

In [33]:
soup

<!-- \views\_layouts\wp2\index.php -->
<!DOCTYPE html>

<html lang="ru" prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article#">
<head>
<!-- Basic page needs
        ============================================ -->
<title>Котировки онлайн: валюты, акции, с международной валютной биржи и ММВБ - ForTrader.org</title>
<link href="/assets/1193a158/jquery.arcticmodal-0.3.css?8cacec34" rel="stylesheet" type="text/css"/>
<style type="text/css">
/*<![CDATA[*/

		a.decomments-button.decomments-button-reply{
			display:none;
		}
	
/*]]>*/
</style>
<script src="/assets/ebd9eaf7/jquery.min.js?ea72fe5a" type="text/javascript"></script>
<script src="/assets/ebd9eaf7/jquery.cookie.js?e085a6dc" type="text/javascript"></script>
<script src="/assets/8779a3f9/wSearchQuotesWidget.js?21ca10d3" type="text/javascript"></script>
<script src="/assets/2d11baaf/autobahn.min.js?8cacec34" type="text/javascript"></script>
<script src="/assets/8779a3f9/wNewQuotesWidget.js?2843381e" type="text/javascript"><

In [34]:
soup.find_all('table')[0]


<table>
<thead>
<tr><th>Название</th><th>Цена</th><th>Изменение</th><th>Изменение(%)</th><th>Обновлено</th></tr>
</thead>
<tbody>
<tr class="" data-symbol="EURRUR">
<td class="tabl_quotes_tipe">
<span class="p_rel blue_color">
<a href="/quotes/eurrur">
											EURRUB											<span class="tooltip">Курс евро к рублю сегодня</span> </a>
</span>
</td>
<td class="red_color pid-EURRUR-bid" data-last-bid="86.564" data-precision="2" data-title="Last">
									86.11								</td>
<td class="red_color pid-EURRUR-chg" data-title="Chg.">
									-0.46								</td>
<td class="red_color pid-EURRUR-chg-percent" data-title="Chg.%">
									-0.53								</td>
<td class="pid-EURRUR-time pid-time" data-start-time="21:50:00" data-title="Time"></td>
</tr>
<tr class="" data-symbol="USDRUR">
<td class="tabl_quotes_tipe">
<span class="p_rel blue_color">
<a href="/quotes/usdrur">
											USDRUB											<span class="tooltip">Курс рубля к доллару</span> </a>
</span>
</td>
<td class="red_color pid-USD

In [35]:
fortrader_df = pd.read_html('https://fortrader.org/quotes')[0]
fortrader_df.head()

,Название,Цена,Изменение,Изменение(%),Обновлено
0,EURRUB Курс евро к рублю сегодня,86.11,-0.46,-0.53,NaN
1,USDRUB Курс рубля к доллару,76.24,-0.12,-0.15,NaN


In [36]:
soup.find_all('table')[1]


<table>
<thead>
<tr><th>Название</th><th>Цена</th><th>Изменение</th><th>Изменение(%)</th><th>Обновлено</th></tr>
</thead>
<tbody>
<tr class="" data-symbol="brentoil">
<td class="tabl_quotes_tipe">
<span class="p_rel blue_color">
<a href="/quotes/brentoil">
											Нефть Brent											<span class="tooltip">Курс нефти сегодня</span> </a>
</span>
</td>
<td class="red_color pid-brentoil-bid" data-last-bid="80.05" data-precision="2" data-title="Last">
									82.09								</td>
<td class="green_color pid-brentoil-chg" data-title="Chg.">
									+2.04								</td>
<td class="green_color pid-brentoil-chg-percent" data-title="Chg.%">
									+2.55								</td>
<td class="pid-brentoil-time pid-time" data-start-time="21:50:00" data-title="Time"></td>
</tr>
<tr class="" data-symbol="wtioil">
<td class="tabl_quotes_tipe">
<span class="p_rel blue_color">
<a href="/quotes/wtioil">
											Нефть WTI											<span class="tooltip">Цены на нефть WTI</span> </a>
</span>
</td>
<td class="red_

In [37]:
fortrader_df = pd.read_html('https://fortrader.org/quotes')[1]
fortrader_df.head()

,Название,Цена,Изменение,Изменение(%),Обновлено
0,Нефть Brent Курс нефти сегодня,82.09,2.04,2.55,NaN
1,Нефть WTI Цены на нефть WTI,79.68,2.48,3.21,NaN
2,Природный газ Цена на природный газ,3.85,-0.01,-0.21,NaN
3,Кофе,231.85,-0.15,-0.06,NaN
4,Кукуруза,604.00,1.60,0.27,NaN


In [38]:
soup.find_all('table')[2]


<table>
<thead>
<tr><th>Название</th><th>Цена</th><th>Изменение</th><th>Изменение(%)</th><th>Обновлено</th></tr>
</thead>
<tbody>
<tr class="" data-symbol="Gold">
<td class="tabl_quotes_tipe">
<span class="p_rel blue_color">
<a href="/quotes/gold">
											Золото											<span class="tooltip">Цена золота на бирже</span> </a>
</span>
</td>
<td class="red_color pid-Gold-bid" data-last-bid="1812.1" data-precision="0" data-title="Last">
									1 790								</td>
<td class="red_color pid-Gold-chg" data-title="Chg.">
									-22								</td>
<td class="red_color pid-Gold-chg-percent" data-title="Chg.%">
									-1								</td>
<td class="pid-Gold-time pid-time" data-start-time="20:49:52" data-title="Time"></td>
</tr>
<tr class="" data-symbol="Silver">
<td class="tabl_quotes_tipe">
<span class="p_rel blue_color">
<a href="/quotes/silver">
											Серебро											<span class="tooltip">Цена на серебро на бирже</span> </a>
</span>
</td>
<td class="green_color pid-Silver-bid" data-l

In [39]:
fortrader_df = pd.read_html('https://fortrader.org/quotes')[2]
fortrader_df.head()

,Название,Цена,Изменение,Изменение(%),Обновлено
0,Золото Цена золота на бирже,1 790,-22.00,-1.00,NaN
1,Серебро Цена на серебро на бирже,22.17,-0.60,-2.65,NaN
2,Палладий Цена на Палладий,1 870,7.00,0.00,NaN
3,Платина Цены на платину,964,-21.00,-2.00,NaN
4,Медь Цена меди на бирже,4.35,-0.03,-0.74,NaN
